In [21]:
import pandas as pd
from pm4py.objects.conversion.log import converter as log_converter
from pm4py.objects.log.importer.xes import importer as xes_importer

In [2]:
log = xes_importer.apply('data/PurchasingExample.xes')
event_log = log_converter.apply(log, variant=log_converter.Variants.TO_DATA_FRAME)

parsing log, completed traces ::   0%|          | 0/608 [00:00<?, ?it/s]

In [3]:
event_log_by_case = event_log.groupby(by='case:concept:name')
event_log_by_case.groups.keys()

dict_keys(['1', '10', '100', '1004', '1008', '1009', '101', '1011', '1015', '1018', '102', '1020', '1021', '1022', '1023', '1025', '103', '1031', '1033', '1035', '104', '1041', '1042', '1044', '1047', '1048', '1052', '1057', '106', '1060', '1063', '1065', '1066', '1068', '107', '1079', '1080', '1085', '1088', '109', '1090', '1094', '11', '110', '1104', '111', '1111', '1114', '1116', '1118', '112', '1123', '1128', '1141', '1144', '1146', '115', '1151', '1153', '1158', '1160', '1161', '117', '1177', '1178', '1179', '118', '1182', '1184', '1187', '119', '1191', '1192', '1197', '1198', '12', '120', '1212', '1214', '1215', '122', '1220', '1225', '1227', '123', '124', '1242', '125', '1255', '1259', '127', '128', '1282', '1284', '1287', '129', '1290', '1294', '1298', '1299', '13', '1300', '1305', '1308', '1309', '1310', '132', '1328', '133', '1337', '1338', '134', '1347', '135', '1353', '1358', '136', '1360', '1361', '1362', '1366', '1367', '1368', '137', '1375', '1377', '1378', '138', '1380'

In [5]:
case = event_log_by_case.get_group('10')
case.head()

,concept:name,lifecycle:transition,org:resource,time:timestamp,Activity,Resource,case:concept:name,case:variant,case:variant-index,case:creator
344,Create Purchase Requisition,start,Esmana Liubiata,2011-01-02 02:52:00-05:00,Create Purchase Requisition,Esmana Liubiata,10,Variant 9,9,Fluxicon Disco
345,Create Purchase Requisition,complete,Esmana Liubiata,2011-01-02 03:43:00-05:00,Create Purchase Requisition,Esmana Liubiata,10,Variant 9,9,Fluxicon Disco
346,Create Request for Quotation,start,Immanuel Karagianni,2011-01-02 10:25:00-05:00,Create Request for Quotation,Immanuel Karagianni,10,Variant 9,9,Fluxicon Disco
347,Create Request for Quotation,complete,Immanuel Karagianni,2011-01-02 10:32:00-05:00,Create Request for Quotation,Immanuel Karagianni,10,Variant 9,9,Fluxicon Disco
348,Analyze Request for Quotation,start,Francois de Perrier,2011-01-02 15:32:00-05:00,Analyze Request for Quotation,Francois de Perrier,10,Variant 9,9,Fluxicon Disco


In [6]:
case = case.sort_values(by='time:timestamp')
case.head()

,concept:name,lifecycle:transition,org:resource,time:timestamp,Activity,Resource,case:concept:name,case:variant,case:variant-index,case:creator
344,Create Purchase Requisition,start,Esmana Liubiata,2011-01-02 02:52:00-05:00,Create Purchase Requisition,Esmana Liubiata,10,Variant 9,9,Fluxicon Disco
345,Create Purchase Requisition,complete,Esmana Liubiata,2011-01-02 03:43:00-05:00,Create Purchase Requisition,Esmana Liubiata,10,Variant 9,9,Fluxicon Disco
346,Create Request for Quotation,start,Immanuel Karagianni,2011-01-02 10:25:00-05:00,Create Request for Quotation,Immanuel Karagianni,10,Variant 9,9,Fluxicon Disco
347,Create Request for Quotation,complete,Immanuel Karagianni,2011-01-02 10:32:00-05:00,Create Request for Quotation,Immanuel Karagianni,10,Variant 9,9,Fluxicon Disco
348,Analyze Request for Quotation,start,Francois de Perrier,2011-01-02 15:32:00-05:00,Analyze Request for Quotation,Francois de Perrier,10,Variant 9,9,Fluxicon Disco


In [26]:
resource_changed = case['Resource'] != case.shift(-1)['Resource']
activity_changed = case['Activity'] != case.shift(-1)['Activity']

# because of NaN at the end of the shifted dataframe, we always have True
resource_changed.iloc[-1] = False
activity_changed.iloc[-1] = False

# both conditions must be satisfied
handoff_occurred = resource_changed & activity_changed

case['handoff_occurred'] = handoff_occurred
case[['case:concept:name', 'Activity', 'Resource', 'handoff_occurred']]

344    False
345     True
346    False
347     True
348    False
349     True
350    False
351    False
352    False
353     True
354    False
355     True
356    False
357     True
358    False
359    False
360    False
361     True
362    False
363     True
364    False
365     True
366    False
367     True
368    False
369     True
371    False
370     True
372    False
373    False
374    False
375    False
376    False
377     True
378    False
379    False
dtype: bool

In [9]:
# case['handoff_frequency'] = 0
case.loc[case['handoff_occurred'] == True, 'handoff_frequency'] = 1
case.head()

,concept:name,lifecycle:transition,org:resource,time:timestamp,Activity,Resource,case:concept:name,case:variant,case:variant-index,case:creator,handoff_occurred,handoff_frequency
344,Create Purchase Requisition,start,Esmana Liubiata,2011-01-02 02:52:00-05:00,Create Purchase Requisition,Esmana Liubiata,10,Variant 9,9,Fluxicon Disco,False,NaN
345,Create Purchase Requisition,complete,Esmana Liubiata,2011-01-02 03:43:00-05:00,Create Purchase Requisition,Esmana Liubiata,10,Variant 9,9,Fluxicon Disco,True,1.0
346,Create Request for Quotation,start,Immanuel Karagianni,2011-01-02 10:25:00-05:00,Create Request for Quotation,Immanuel Karagianni,10,Variant 9,9,Fluxicon Disco,False,NaN
347,Create Request for Quotation,complete,Immanuel Karagianni,2011-01-02 10:32:00-05:00,Create Request for Quotation,Immanuel Karagianni,10,Variant 9,9,Fluxicon Disco,True,1.0
348,Analyze Request for Quotation,start,Francois de Perrier,2011-01-02 15:32:00-05:00,Analyze Request for Quotation,Francois de Perrier,10,Variant 9,9,Fluxicon Disco,False,NaN


In [10]:
handoff_start = case.loc[handoff_occurred, 'time:timestamp']
handoff_end = case.loc[handoff_occurred.shift(1, fill_value=False), 'time:timestamp']
case.loc[handoff_occurred, 'handoff_duration'] = handoff_end.values - handoff_start.values
case.head()

,concept:name,lifecycle:transition,org:resource,time:timestamp,Activity,Resource,case:concept:name,case:variant,case:variant-index,case:creator,handoff_occurred,handoff_frequency,handoff_duration
344,Create Purchase Requisition,start,Esmana Liubiata,2011-01-02 02:52:00-05:00,Create Purchase Requisition,Esmana Liubiata,10,Variant 9,9,Fluxicon Disco,False,NaN,NaT
345,Create Purchase Requisition,complete,Esmana Liubiata,2011-01-02 03:43:00-05:00,Create Purchase Requisition,Esmana Liubiata,10,Variant 9,9,Fluxicon Disco,True,1.0,0 days 06:42:00
346,Create Request for Quotation,start,Immanuel Karagianni,2011-01-02 10:25:00-05:00,Create Request for Quotation,Immanuel Karagianni,10,Variant 9,9,Fluxicon Disco,False,NaN,NaT
347,Create Request for Quotation,complete,Immanuel Karagianni,2011-01-02 10:32:00-05:00,Create Request for Quotation,Immanuel Karagianni,10,Variant 9,9,Fluxicon Disco,True,1.0,0 days 05:00:00
348,Analyze Request for Quotation,start,Francois de Perrier,2011-01-02 15:32:00-05:00,Analyze Request for Quotation,Francois de Perrier,10,Variant 9,9,Fluxicon Disco,False,NaN,NaT


In [11]:
case['handoff_score'] = case['handoff_duration'] * case['handoff_frequency']
case

,concept:name,lifecycle:transition,org:resource,time:timestamp,Activity,Resource,case:concept:name,case:variant,case:variant-index,case:creator,handoff_occurred,handoff_frequency,handoff_duration,handoff_score
344,Create Purchase Requisition,start,Esmana Liubiata,2011-01-02 02:52:00-05:00,Create Purchase Requisition,Esmana Liubiata,10,Variant 9,9,Fluxicon Disco,False,NaN,NaT,NaT
345,Create Purchase Requisition,complete,Esmana Liubiata,2011-01-02 03:43:00-05:00,Create Purchase Requisition,Esmana Liubiata,10,Variant 9,9,Fluxicon Disco,True,1.0,0 days 06:42:00,0 days 06:42:00
346,Create Request for Quotation,start,Immanuel Karagianni,2011-01-02 10:25:00-05:00,Create Request for Quotation,Immanuel Karagianni,10,Variant 9,9,Fluxicon Disco,False,NaN,NaT,NaT
347,Create Request for Quotation,complete,Immanuel Karagianni,2011-01-02 10:32:00-05:00,Create Request for Quotation,Immanuel Karagianni,10,Variant 9,9,Fluxicon Disco,True,1.0,0 days 05:00:00,0 days 05:00:00
348,Analyze Request for Quotation,start,Francois de Perrier,2011-01-02 15:32:00-05:00,Analyze Request for Quotation,Francois de Perrier,10,Variant 9,9,Fluxicon Disco,False,NaN,NaT,NaT
349,Analyze Request for Quotation,complete,Francois de Perrier,2011-01-02 15:59:00-05:00,Analyze Request for Quotation,Francois de Perrier,10,Variant 9,9,Fluxicon Disco,True,1.0,0 days 20:51:00,0 days 20:51:00
350,Send Request for Quotation to Supplier,start,Karel de Groot,2011-01-03 12:50:00-05:00,Send Request for Quotation to Supplier,Karel de Groot,10,Variant 9,9,Fluxicon Disco,False,NaN,NaT,NaT
351,Send Request for Quotation to Supplier,complete,Karel de Groot,2011-01-03 13:07:00-05:00,Send Request for Quotation to Supplier,Karel de Groot,10,Variant 9,9,Fluxicon Disco,False,NaN,NaT,NaT
352,Create Quotation comparison Map,start,Karel de Groot,2011-01-03 13:31:00-05:00,Create Quotation comparison Map,Karel de Groot,10,Variant 9,9,Fluxicon Disco,False,NaN,NaT,NaT
353,Create Quotation comparison Map,complete,Karel de Groot,2011-01-03 14:28:00-05:00,Create Quotation comparison Map,Karel de Groot,10,Variant 9,9,Fluxicon Disco,True,1.0,0 days 07:16:00,0 days 07:16:00


In [12]:
for (activity, group) in case[case['handoff_score'].notna()].groupby(by='Activity'):
    score_per_activity = group['handoff_score'].sum()
    print(f'{activity}: \t{score_per_activity}')

Analyze Quotation Comparison Map: 	0 days 00:00:00
Analyze Request for Quotation: 	0 days 20:51:00
Approve Purchase Order for payment: 	0 days 21:27:00
Authorize Supplier's Invoice payment: 	0 days 00:20:00
Choose best option: 	0 days 01:48:00
Confirm Purchase Order: 	0 days 08:50:00
Create Purchase Order: 	0 days 16:57:00
Create Purchase Requisition: 	0 days 06:42:00
Create Quotation comparison Map: 	0 days 07:16:00
Create Request for Quotation: 	0 days 05:00:00
Deliver Goods Services: 	0 days 07:35:00
Release Purchase Order: 	1 days 18:00:00
Send Invoice: 	0 days 14:22:00
